In [1]:
import pandas as pd
import numpy as np
import time

from utils.conditioning import get_conditioning_deltas
from utils.routing import get_route_factor_adjustments, get_route_number
from utils.packaging import get_packaging_changes, construct_package_routes

In [2]:
filename = "packages_input.xlsx"
data = pd.read_excel(filename, skiprows=[0], sheet_name="Data")
routing_factors = pd.read_excel(filename, sheet_name="Routing").fillna(0)
packages = pd.read_excel(filename, sheet_name="Packages", skiprows=[0])
rads = pd.read_excel(filename, sheet_name="Rads", skiprows=[0])

radionucleide_column_indices = range(9, 12) # 0-indexed column numbers for any radionucleide activity values
radionucleide_cols = [data.columns[c] for c in radionucleide_column_indices]

# The packages sheet can be simplified by taking only the primary package definitions and defining secondary state packaging seperately
packages = packages[list(packages.columns[:8]) + [packages.columns[-1]]]
package_routes = [("Drum", "Stillage"),
                  ("Shielded drum", "No Package"),
                  ("4m box", "No Package"),
                  ("3m box", "No Package"),
                  ("No Package", "No Package"),
                  ("Stillage", "No Package")]

package_routes_df = construct_package_routes(package_routes, packages)

Pre-calculate the stream route number and mass for convenience

In [3]:
data

,Waste Stream,Density,Volume,Package type,Packaging factor,Conditioning factor,Steel %,Not steel %,Haz %,Am-241,C-60,Zn-65,Conditioning material
0,1A01_2_P,0.50,52.5,NaN,NaN,NaN,0.1,99.9,0.2,0.100000,NaN,NaN,NaN
1,1A07_10_P,0.35,168.5,Drum,1.214894,1.000000,10.0,91.0,20.0,0.639000,1.679250e-03,2.613990e-07,NaN
2,2C05_10_P,1.30,15.0,Shielded drum,1.056000,0.392000,90.0,10.0,3.0,0.008877,1.227000e-02,3.565000e-08,NaN
3,2C38_10_P,0.30,18.6,4m box,1.234568,1.166667,89.0,10.0,4.0,NaN,6.134000e-02,1.780000e-06,NaN
4,7A13_10_P,1.47,476.5,3m box,1.323887,1.000000,50.0,50.0,1.0,0.002654,9.025440e-07,NaN,NaN


In [6]:
data["Route Number"] = get_route_number(data["Waste Stream"])
data["Mass"] = data["Density"] * data["Volume"]
data.fillna({c: 0 for c in radionucleide_cols}, inplace=True)

unpackaged_type = "No Package"
data["Package type"].fillna(unpackaged_type, inplace=True)

# Any remaining null conditioning factors must be set according to package type
default_factors = data.merge(packages, left_on="Package type", right_on="Package type", how="left")[["Default conditioning factor", "Default packing factor"]]
data["Conditioning factor"].fillna(default_factors["Default conditioning factor"], inplace=True)

# Any null packaging factors must be set according to package type
data["Packaging factor"].fillna(default_factors["Default packing factor"], inplace=True)

data

,Waste Stream,Density,Volume,Package type,Packaging factor,Conditioning factor,Steel %,Not steel %,Haz %,Am-241,C-60,Zn-65,Conditioning material,Route Number,Mass
0,1A01_2_P,0.50,52.5,No Package,1.000000,1.000000,0.1,99.9,0.2,0.100000,0.000000e+00,0.000000e+00,NaN,2,26.250
1,1A07_10_P,0.35,168.5,Drum,1.214894,1.000000,10.0,91.0,20.0,0.639000,1.679250e-03,2.613990e-07,NaN,10,58.975
2,2C05_10_P,1.30,15.0,Shielded drum,1.056000,0.392000,90.0,10.0,3.0,0.008877,1.227000e-02,3.565000e-08,NaN,10,19.500
3,2C38_10_P,0.30,18.6,4m box,1.234568,1.166667,89.0,10.0,4.0,0.000000,6.134000e-02,1.780000e-06,NaN,10,5.580
4,7A13_10_P,1.47,476.5,3m box,1.323887,1.000000,50.0,50.0,1.0,0.002654,9.025440e-07,0.000000e+00,NaN,10,700.455


Determine the routing adjustments for the streams:

In [5]:
start = time.time()

route_adjusted_data = data.copy(deep=True) # make a copy so we can preserve output at each step

route_adjustment_values = get_route_factor_adjustments(streams=data, route_factors=routing_factors, radionucleide_cols=radionucleide_cols)
route_adjusted_data[route_adjustment_values.columns] = route_adjustment_values
route_adjusted_data

,Waste Stream,Density,Volume,Package type,Packaging factor,Conditioning factor,Steel %,Not steel %,Haz %,Am-241,C-60,Zn-65,Conditioning material,Route Number,Mass
0,1A01_2_P,0.50,52.5,No Package,1.000000,1.000000,0.1,99.9,0.2,0.000000,0.000000e+00,0.000000e+00,NaN,2,26.250
1,1A07_10_P,0.35,168.5,Drum,1.214894,1.000000,10.0,91.0,20.0,0.639000,1.679250e-03,2.613990e-07,NaN,10,58.975
2,2C05_10_P,1.30,15.0,Shielded drum,1.056000,0.392000,90.0,10.0,3.0,0.008877,1.227000e-02,3.565000e-08,NaN,10,19.500
3,2C38_10_P,0.30,18.6,4m box,1.234568,1.166667,89.0,10.0,4.0,0.000000,6.134000e-02,1.780000e-06,NaN,10,5.580
4,7A13_10_P,1.47,476.5,3m box,1.323887,1.000000,50.0,50.0,1.0,0.002654,9.025440e-07,0.000000e+00,NaN,10,700.455


In [6]:
conditional_changes = get_conditioning_deltas(route_adjusted_data)

conditioned_data = route_adjusted_data.copy(deep=True)
conditioned_data["Volume"] += conditional_changes["Volume Delta"]
conditioned_data["Mass"] += conditional_changes["Mass Delta"]
conditioned_data["Conditioning material"] = conditional_changes["Conditioning material"]
conditioned_data

,Waste Stream,Density,Volume,Package type,Packaging factor,Conditioning factor,Steel %,Not steel %,Haz %,Am-241,C-60,Zn-65,Conditioning material,Route Number,Mass
0,1A01_2_P,0.50,52.50,No Package,1.000000,1.000000,0.1,99.9,0.2,0.000000,0.000000e+00,0.000000e+00,No Conditioning Material,2,26.250
1,1A07_10_P,0.35,168.50,Drum,1.214894,1.000000,10.0,91.0,20.0,0.639000,1.679250e-03,2.613990e-07,No Conditioning Material,10,58.975
2,2C05_10_P,1.30,5.88,Shielded drum,1.056000,0.392000,90.0,10.0,3.0,0.008877,1.227000e-02,3.565000e-08,No Conditioning Material,10,19.500
3,2C38_10_P,0.30,21.70,4m box,1.234568,1.166667,89.0,10.0,4.0,0.000000,6.134000e-02,1.780000e-06,Grout,10,10.850
4,7A13_10_P,1.47,476.50,3m box,1.323887,1.000000,50.0,50.0,1.0,0.002654,9.025440e-07,0.000000e+00,No Conditioning Material,10,700.455


In [7]:
packing_changes = get_packaging_changes(conditioned_data, package_routes_df)

packed_data = conditioned_data.copy(deep=True)
packed_data["Volume"] = packing_changes["Secondary Packaged Volume"]
packed_data["Mass"] = packed_data["Mass"] + packing_changes["Primary Mass Delta"] + packing_changes["Secondary Mass Delta"]
# optional -overwrite package type to avoid misleading data

packed_data

,Waste Stream,Density,Volume,Package type,Packaging factor,Conditioning factor,Steel %,Not steel %,Haz %,Am-241,C-60,Zn-65,Conditioning material,Route Number,Mass
0,1A01_2_P,0.50,52.500000,No Package,1.000000,1.000000,0.1,99.9,0.2,0.000000,0.000000e+00,0.000000e+00,No Conditioning Material,2,26.250000
1,1A07_10_P,0.35,304.880516,Drum,1.214894,1.000000,10.0,91.0,20.0,0.639000,1.679250e-03,2.613990e-07,No Conditioning Material,10,184.866363
2,2C05_10_P,1.30,15.840000,Shielded drum,1.056000,0.392000,90.0,10.0,3.0,0.008877,1.227000e-02,3.565000e-08,No Conditioning Material,10,85.558546
3,2C38_10_P,0.30,22.962963,4m box,1.234568,1.166667,89.0,10.0,4.0,0.000000,6.134000e-02,1.780000e-06,Grout,10,16.184238
4,7A13_10_P,1.47,630.831984,3m box,1.323887,1.000000,50.0,50.0,1.0,0.002654,9.025440e-07,0.000000e+00,No Conditioning Material,10,837.472944


(Emplacement vol - External vol) / no. of packages